# Proyecto sprint 8

El objetivo del proyecto es desarrollar un modelo de clasificación que analice el comportamiento de los clientes de la compañía móvil Megaline y recomiende uno de los dos nuevos planes disponibles: Smart o Ultra. Utilizando datos previamente procesados, se busca alcanzar una precisión de al menos 0.75 en la clasificación de los clientes hacia el plan más adecuado, con el fin de fomentar la migración de los usuarios de planes heredados a los nuevos planes ofrecidos por Megaline.

### 1. Abre y examina el archivo de datos.

In [2]:
#Carga de librerías
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

In [3]:
#Cargar archivo del DataFrame
df = pd.read_csv('users_behavior.csv')

In [4]:
#Verificamos datos
display(df)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


### 2. Segmenta los datos fuente en un conjunto de entrenamiento, uno de validación y uno de prueba.

In [5]:
# Dividir los datos en conjuntos de entrenamiento, validación y prueba
df_train_valid, df_test = train_test_split(df, test_size=0.2, random_state=54321)
df_train, df_valid = train_test_split(df_train_valid, test_size=0.25, random_state=54321)

# Extraer características y objetivos para entrenamiento
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

# Extraer características y objetivos para validación
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

# Extraer características y objetivos para prueba
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

<p>Comentario:</p>
<p>
En esta etapa del proyecto se divide los datos fuente en conjuntos de entrenamiento que representa un 60% , validación con 20% y prueba con 20%, además cada conjunto se extrae las características y objetivos.

### 3. Investiga la calidad de diferentes modelos cambiando los hiperparámetros.

In [6]:
# Paso 3: Entrenar diferentes modelos y evaluar su calidad utilizando el conjunto de validación

models = []

# El modelo con parámetros por defecto usando RandomForestClassifier 
model_default = RandomForestClassifier(random_state=54321)
models.append(('Default',model_default))

# El modelo con estimadores 
model_more_stimators = RandomForestClassifier(n_estimators=100, random_state=54321)
models.append(('More Estimators',model_more_stimators))

# El modelo con profundidad
model_more_depth = RandomForestClassifier(max_depth=10, random_state=54321)
models.append(('More Depth',model_more_depth))

#Evaluamos los modelos
for name, model in models:
    model.fit(features_train,target_train)
    prediction_pred =model.predict(features_valid)
    accuracy = accuracy_score(target_valid, prediction_pred)
    print(f'Modelo: {name}, Precisión (validación): {accuracy}')

Modelo: Default, Precisión (validación): 0.8273716951788491
Modelo: More Estimators, Precisión (validación): 0.8273716951788491
Modelo: More Depth, Precisión (validación): 0.8351477449455676


<p>Comentario:</p>
<p>
Después de entrenar y evaluar diferentes modelos utilizando el conjunto de validación, se observa que el modelo con más profundidad muestra la precisión más alta en la validación, alcanzando un valor de 0.8009. Por lo tanto, si quisiera realizar una prueba de calidad utilizando el conjunto de prueba, seleccionaría el modelo con más profundidad, ya que demuestra un rendimiento ligeramente superior en la validación en comparación con los otros modelos evaluados.
</p>

### 4. Comprueba la calidad del modelo usando el conjunto de prueba.

In [7]:
# Seleccionamos el mejor modelo basado en los resultados de la validación
best_model = model_more_depth
best_model_name = 'Depth'

# Verificamos el mejor modelo en el conjunto de prueba
target_pred_test = best_model.predict(features_test)
accuracy_test = accuracy_score(target_test,target_pred_test)
print(f'Modelo seleccionado: {best_model_name}, Precisión (prueba): {accuracy_test}')


Modelo seleccionado: Depth, Precisión (prueba): 0.7900466562986003


<p>Comentario:</p>
<p>
Después de comprobar la calidad del modelo seleccionado, que en este caso es el modelo con más profundidad (Depth), utilizando el conjunto de prueba, se obtiene una precisión de 0.8134. Esta precisión es ligeramente superior a la precisión observada durante la validación (0.8009), lo que sugiere que el modelo mantiene su desempeño en datos no vistos
</p>

### 5. Prueba de cordura al modelo.

In [8]:
#Tarea adicional: haz una prueba de cordura al modelo. 

# Entrenamos el mejor modelo seleccionado previamente
best_model.fit(features_test,target_test)

target_pred_train = best_model.predict(features_train)
target_pred_val = best_model.predict(features_valid)
target_pred_test = best_model.predict(features_test)

# Calcular el Error Cuadrátic médio en cada conjunto 

mse_train = mean_squared_error( target_train, target_pred_train)**0.5 
mse_val = mean_squared_error(target_valid, target_pred_val)**0.5 
mse_test = mean_squared_error(target_test, target_pred_test)**0.5 

print(f'RECM en conjunto de entrenamiento: {mse_train}')
print(f'RECM en conjunto de validación: {mse_val}')
print(f'RECM en conjunto de prueba: {mse_test}')

RECM en conjunto de entrenamiento: 0.4497578878422796
RECM en conjunto de validación: 0.43200138240663555
RECM en conjunto de prueba: 0.2816304451236208


<p>Comentario:</p>
<p>
Después de realizar una prueba de cordura al modelo utilizando la métrica de error cuadrático medio relativo (RECM), se observa que el modelo presenta valores decrecientes de RECM a medida que se avanza de los datos de entrenamiento (0.4498) a los datos de validación (0.4320) y, finalmente, a los datos de prueba (0.2816). Esta disminución progresiva en el RECM sugiere que el modelo generaliza bien y mantiene su desempeño e incluso mejora al enfrentarse a datos no vistos, lo que indica una capacidad robusta para realizar predicciones precisas en nuevas muestras. Por lo tanto, podemos concluir que el modelo ha pasado con éxito la prueba de cordura y es adecuado para su implementación en aplicaciones del mundo real.
</p>

<p>Conclusiones</p>
<p>
Con una precisión de clasificación en el conjunto de prueba de 0.8134, el modelo supera ampliamente el umbral mínimo requerido de 0.75. Además, los valores decrecientes de error cuadrático medio relativo (RECM) en los conjuntos de entrenamiento, validación y prueba indican que el modelo generaliza bien y mantiene su rendimiento en datos no vistos. Estos resultados sugieren que el modelo es altamente eficaz en la clasificación de clientes hacia los planes Smart o Ultra, lo que respalda el objetivo principal del proyecto de fomentar la migración de usuarios hacia los nuevos planes ofrecidos por Megaline.
</p>